# Emluate Pytorch Autograd with Custom Code

ref: [Tutorial: Learning Pytroch with Examples](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)

This demonstration aims to explain how autograd in Pytorch works and the purpose of it by emulating it with self-explanitory code. In our example, we wish to determine the coefficients of the series expansion of $sin(x)$ using the gradient descent method. 

A polynomial expansion of a function $f$ is defined as:

\begin{equation}

f(x) = \sum^{\infty}_{k=0}{w_k x^k} = w_0 + w_1x + w_2x^2 + w_3x^3 + w_4x^4 + w_5x^5 .....

\end{equation}

This series expansion should have infinite number of terms, but for physically feasible scenerios, ususally $f(x)$ is approximated as the sum of the first $n$ terms, where $n \in N$. This is called the $n$ term approximation of function $f(x)$.

\begin{equation}
f(x) \approx \sum^{n}_{k=0}{w_k x_k} = w_0 + w_1x + w_2x^2 + w_3x^3 + w_4x^4 ..... + w_kx_k
\end{equation}

Although usually there are mathematical solutions to find the coefficients (we will call them weights in this demo) $w_k$, but for the sake of this demonstration, we attempt to calculate the weights $w_k$ in the polynomial expansion of $sin(x)$ using gradient descent. To measure how good the approximation fits the actual value of $sin(x)$, we use mean square loss as the loss function, defined as 

\begin{equation}
L = (\hat{y} - y)^2 \quad \text{$\hat{y}$: true value of $sin(x)$}
\end{equation}

Taking the partial derivative of $L$ with respect to each weight, the gradient of a weight can be found
\begin{equation}
\frac{\partial L}{\partial w_k} = \frac{\partial (\hat{y}-y)^2}{\partial w_k} = 2(\hat{y}-y)x^k
\end{equation}

And when the gradient of a weight $w_k$ is near 0, the optimal value of that weight is reached.

\begin{equation}
\frac{\partial L}{\partial w_k} \approx 0
\end{equation}

First, let's start with the ```numpy``` library.

## Gradient Descent with Numpy

In [ ]:
import numpy as np

N_SAMPLES = 4000
LEARNING_RATE = 1e-6

# Create random input and output data
x = np.linspace(-np.pi, np.pi, N_SAMPLES)
y = np.sin(x)

# Randomly initialize weights
np.random.seed(0)
w = np.random.random(8)

for t in range(4000):

    # shape [4000]
    y_pred = w[0] + w[1]*x + w[2]*(x**2) + w[3]*(x**3) + w[4]*(x**4) + w[5]*(x**5) + w[6]*(x**6) + w[7]*(x**7)

    # shape [4000]
    error = y_pred - y

    # shape [1]
    loss = (error ** 2).mean()

    #shape [8]
    gradients = np.array([2 * error * (x**i) for i in range(8)]).mean(-1)
    w -= gradients * LEARNING_RATE
    
    if t % 200 == 199:
        print(f"Loss at {t+1:> 5} iteration: {loss:> 5.3f}")

The library ```Numpy``` performs great! But it usually runs on CPU and requires some additional work to enable GPU calculation of numpy code. ```Pytorch``` framework, on the other hand, is designed to run on cuda-enabled GPUs by default. With good program design, large-scale, computation-intensive tensor operations *should perform better on GPUs than on CPUs. The following are some code snippets written in Pytorch framework that attempts to achieve the same goal as the previous numpy code.

## Red-Neck Gradient Descent in Pytorch

In [ ]:
import torch

dtype = torch.float
device = torch.device("cuda")
torch.manual_seed(0)

N_SAMPLES = 4000
LEARNING_RATE = 1e-6

x = torch.linspace(-torch.pi, torch.pi, N_SAMPLES, device=device, dtype=dtype)
y = torch.sin(x)
w = torch.randn(8, device = device, dtype = dtype)

for t in range(4000):
    # shape [4000]
    y_pred = w[0] + w[1]*x + w[2]*x**2 + w[3]*x**3 + w[4]*x**4 + w[5]*x**5 + w[6]*x**6 + w[7]*x**7

    # shape [4000]
    error = y_pred - y

    # shape [1]
    loss = error.pow(2).mean().item()

    # shape [8]
    gradients = torch.empty_like(w, device = device)
    gradients[0] = (2 * error).mean(-1)
    gradients[1] = (2 * error * x).mean(-1)
    gradients[2] = (2 * error * (x**2)).mean(-1)
    gradients[3] = (2 * error * (x**3)).mean(-1)
    gradients[4] = (2 * error * (x**4)).mean(-1)
    gradients[5] = (2 * error * (x**5)).mean(-1)
    gradients[6] = (2 * error * (x**6)).mean(-1)
    gradients[7] = (2 * error * (x**7)).mean(-1)

    """
    ValueError: only one element tensors can be converted to Python scalars
    gradients = torch.tensor([2 * error * (x**i) for i in range(8)], device = device).mean()
    """

    """ 
    ValueError: only one element tensors can be converted to Python scalars
    gradients = torch.tensor((
        2 * error,
        2 * error * x,
        2 * error * (x**2),
        2 * error * (x**3),
        2 * error * (x**4),
        2 * error * (x**5),
        2 * error * (x**6),
        2 * error * (x**7)), device = device).mean(-1)
    gradients = torch.from_numpy(__gradients, device = device)
    """

    w -= gradients * LEARNING_RATE
    
    if t % 200 == 199:
        print(f"Loss at {t+1:> 5} iteration: {loss:> 5.3f}")


In [ ]:
import torch

dtype = torch.float
device = "cuda"
torch.set_default_device(device)
torch.manual_seed(0)

N_SAMPLES = 4000
LEARNING_RATE = 1e-6

x = torch.linspace(-torch.pi, torch.pi, N_SAMPLES, dtype=dtype)
y = torch.sin(x)
w = torch.randn(8, device = device, dtype = dtype, requires_grad = True)

for t in range(4000):
    y_pred = w[0] + w[1]*x + w[2]*x**2 + w[3]*x** 3 + w[4]*x**4 + w[5]*x**5 + w[6]*x**6 + w[7]*x**7

    error = y_pred - y
    loss = error.pow(2).mean()

    loss.backward()

    with torch.no_grad():
        w -= w.grad * LEARNING_RATE
        w.grad[:] = 0
        
    if t % 200 == 199:
        print(f"Loss at {t+1:> 5} iteration: {loss:> 5.3f}")


## Custom Autograd Functions in Pytorch
### Using Legendre polynomial approximation of sine function as an example.

ref:

[Legendre polynomial -- Academic Accelerator](https://academic-accelerator.com/encyclopedia/legendre-polynomials)

[Legendre polynomial -- Wolfram Mathworld](https://mathworld.wolfram.com/LegendrePolynomial.html)


An arbritary function, $f(x)$, in domain $x \in [-1, 1]$ can be approximated as such:
\begin{equation}
f(x) = \sum^{\infty}_{n=0}{a_n}{P_n} = w_0P_0 + w_1P_1 + w_2P_2 + w_3P_3 + w_4P_4 ..... + w_nP_n
\end{equation}

where $P_n$ is the Legendre polynomial of order $n$ and 
\begin{equation}
w_n = \frac{<f(x), P_n>}{<P_n, P_n>}
\end{equation}

The leading 6 orders of Legendre polynomial are as follows:

\begin{align}
P_0(x) &= 1 \\
P_1(x) &= x \\
P_2(x) &= \frac{1}{2}(3x^2 - 1) \\
P_3(x) &= \frac{1}{2}(5x^3 - 3x) \\
P_4(x) &= \frac{1}{8}(35x^4 - 30x^2 + 3) \\
P_5(x) &= \frac{1}{8}(63x^5 - 70x^3 + 15x) \\
P_6(x) &= \frac{1}{16}(231x^6 - 315x^4 + 105x^2-5) \\
\end{align}

For this demonstration, we wish to find the fifth-order Legendre polynomial approximation of $sin(x)$. The weights, $w_n$, are to be determined using gradient descent. However, since $sin(x)$ is an odd function, we can safely ignore the even order terms and only have to find the coefficients of the odd order terms.

\begin{equation}
sin(x) \approx w_1P_1(x) + w_3P_3(x) + w_5P_5(x)
\end{equation}

Taking partial derivative of the mean square loss function with respect to each weight, the gradient of each weight can be calculated as

\begin{align}
\frac {\partial L}{\partial w_n} &= \frac {\partial (\hat{y} - y)^2}{\partial w_n} \\
&= 2(\hat{y} - y) \frac{\partial \hat{y}}{\partial w_n} \\
&= 2(\hat{y} - y)P_n(x)
\end{align}

In [ ]:
import torch
from LegendrePolynomial import ThirdOrderLegendrePolynomial, FifthOrderLegendrePolynomial

dtype = torch.float
device = torch.device("cuda")
torch.set_default_device(device)
torch.manual_seed(0)

N_SAMPLES = 4000
LEARNING_RATE = 1e-6

x = torch.linspace(-torch.pi, torch.pi, N_SAMPLES, dtype=dtype)
y = torch.sin(x)
w = torch.randn(3, device = device, dtype = dtype, requires_grad = True)

for t in range(4000):

    P3 = ThirdOrderLegendrePolynomial.apply
    P5 = FifthOrderLegendrePolynomial.apply
    
    y_pred = w[0]*x + w[1]*P3(x) + w[2]*P5(x)

    error = y_pred - y
    loss = (error ** 2).mean()

    loss.backward()

    # tempoarily turn off autograd for the operation in this block.
    # manual weight update should not be recorded as part of the operation of finding calculating gradients
    with torch.no_grad():
        w -= w.grad * LEARNING_RATE
        w.grad[:] = 0
        
    if t % 200 == 199:
        print(f"Loss at {t+1:> 5} iteration: {loss:> 5.3f}")